In [1]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

In [2]:
folder_path = '/content/drive/MyDrive/skin_detected'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
spring_tone = [253.75, 153.55, 152]  # 빨간색
summer_tone = [254, 291.25, 164.5]  # 파란색
fall_tone = [240.75, 199.25, 132.75]  # 파란색
winter_tone = [240, 201.5, 129.5]  # 파란색

def zero_shot_classification(color_vector):
    spring_similarity = np.dot(color_vector, spring_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(spring_tone))
    summer_similarity = np.dot(color_vector, summer_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(summer_tone))
    fall_similarity = np.dot(color_vector, fall_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(fall_tone))
    winter_similarity = np.dot(color_vector, winter_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(winter_tone))

    if spring_similarity > summer_similarity and spring_similarity > fall_similarity and spring_similarity > winter_similarity:
        return "봄 웜톤"
    elif summer_similarity > spring_similarity and summer_similarity > fall_similarity and summer_similarity > winter_similarity:
        return "여름 쿨톤"
    elif fall_similarity > spring_similarity and fall_similarity > summer_similarity and fall_similarity > winter_similarity:
        return "가을 웜톤"
    else:
        return "겨울 쿨톤"

In [5]:
def classify_image(image):
    image = cv2.imread(image) ## 함수로 라벨 반환 시 사용
    image = image / 255.0  # Normalize image to 0-1 range
    height, width, _ = image.shape
    colors = np.reshape(image, (height * width, 3))  # Convert all pixels in the image to a 1D vector

    color_labels = []
    for color in colors:
        color_label = zero_shot_classification(color)
        color_labels.append(color_label)

    spring_tone_count = color_labels.count('봄 웜톤')
    summer_tone_count = color_labels.count('여름 쿨톤')
    fall_tone_count = color_labels.count('가을 웜톤')
    winter_tone_count = color_labels.count('겨울 쿨톤')

    total_count = len(color_labels)

    spring_tone_ratio = spring_tone_count / total_count
    summer_tone_ratio = summer_tone_count / total_count
    fall_tone_ratio = fall_tone_count / total_count
    winter_tone_ratio = winter_tone_count / total_count

    ratio_sum = spring_tone_ratio + summer_tone_ratio + fall_tone_ratio + winter_tone_ratio

    spring_tone_ratio = spring_tone_ratio / ratio_sum * 100
    summer_tone_ratio = summer_tone_ratio / ratio_sum * 100
    fall_tone_ratio = fall_tone_ratio / ratio_sum * 100
    winter_tone_ratio = winter_tone_ratio / ratio_sum * 100

    return f"봄 웜톤: {spring_tone_ratio:.2f}%\n여름 쿨톤: {summer_tone_ratio:.2f}%\n가을 웜톤: {fall_tone_ratio:.2f}%\n겨울 쿨톤: {winter_tone_ratio:.2f}%"



In [6]:
from PIL import Image
import os

# 이미지가 있는 폴더 경로
warm_tone_folder = "/content/drive/MyDrive/cropped_face/웜톤"
cool_tone_folder = "/content/drive/MyDrive/cropped_face/쿨톤"

# 이미지를 저장할 리스트
warm_tone_images = []
cool_tone_images = []

# 웜톤 폴더의 이미지 읽기
for filename in os.listdir(warm_tone_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        image_path = os.path.join(warm_tone_folder, filename)
        image = Image.open(image_path)
        warm_tone_images.append(image)

# 쿨톤 폴더의 이미지 읽기
for filename in os.listdir(cool_tone_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        image_path = os.path.join(cool_tone_folder, filename)
        image = Image.open(image_path)
        cool_tone_images.append(image)

# 이미지 개수 확인
warm_tone_count = len(warm_tone_images)
cool_tone_count = len(cool_tone_images)

print(f"웜톤 이미지 개수: {warm_tone_count}")
print(f"쿨톤 이미지 개수: {cool_tone_count}")

웜톤 이미지 개수: 51
쿨톤 이미지 개수: 50


In [7]:
file_results = {}

for filename in os.listdir(warm_tone_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        file_path = os.path.join(warm_tone_folder, filename)
        result = classify_image(file_path)
        file_results[filename] = result

# 결과 출력
for filename, result in file_results.items():
    print(f"{filename} 파일의 결과:\n{result}\n")

<ipython-input-4-3b249918057d>:7: RuntimeWarning: invalid value encountered in double_scalars
  spring_similarity = np.dot(color_vector, spring_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(spring_tone))
<ipython-input-4-3b249918057d>:8: RuntimeWarning: invalid value encountered in double_scalars
  summer_similarity = np.dot(color_vector, summer_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(summer_tone))
<ipython-input-4-3b249918057d>:9: RuntimeWarning: invalid value encountered in double_scalars
  fall_similarity = np.dot(color_vector, fall_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(fall_tone))
<ipython-input-4-3b249918057d>:10: RuntimeWarning: invalid value encountered in double_scalars
  winter_similarity = np.dot(color_vector, winter_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(winter_tone))


하연수.jpg 파일의 결과:
봄 웜톤: 10.38%
여름 쿨톤: 89.57%
가을 웜톤: 0.04%
겨울 쿨톤: 0.00%

한선화.jpg 파일의 결과:
봄 웜톤: 14.26%
여름 쿨톤: 85.67%
가을 웜톤: 0.07%
겨울 쿨톤: 0.00%

제시.jpg 파일의 결과:
봄 웜톤: 33.10%
여름 쿨톤: 65.99%
가을 웜톤: 0.79%
겨울 쿨톤: 0.12%

이효리.jpg 파일의 결과:
봄 웜톤: 8.11%
여름 쿨톤: 91.05%
가을 웜톤: 0.84%
겨울 쿨톤: 0.00%

이민정.jpg 파일의 결과:
봄 웜톤: 5.66%
여름 쿨톤: 94.17%
가을 웜톤: 0.17%
겨울 쿨톤: 0.00%

탕웨이.jpg 파일의 결과:
봄 웜톤: 12.65%
여름 쿨톤: 83.06%
가을 웜톤: 4.29%
겨울 쿨톤: 0.00%

효린.jpeg 파일의 결과:
봄 웜톤: 4.01%
여름 쿨톤: 89.54%
가을 웜톤: 6.45%
겨울 쿨톤: 0.00%

혜리.jpg 파일의 결과:
봄 웜톤: 9.89%
여름 쿨톤: 90.09%
가을 웜톤: 0.00%
겨울 쿨톤: 0.02%

한예슬.jpg 파일의 결과:
봄 웜톤: 26.77%
여름 쿨톤: 72.10%
가을 웜톤: 0.66%
겨울 쿨톤: 0.47%

전지현.jpg 파일의 결과:
봄 웜톤: 50.47%
여름 쿨톤: 49.37%
가을 웜톤: 0.16%
겨울 쿨톤: 0.00%

윤아.jpg 파일의 결과:
봄 웜톤: 21.13%
여름 쿨톤: 78.34%
가을 웜톤: 0.50%
겨울 쿨톤: 0.02%

수애.jpg 파일의 결과:
봄 웜톤: 8.91%
여름 쿨톤: 88.04%
가을 웜톤: 2.61%
겨울 쿨톤: 0.44%

유빈.jpg 파일의 결과:
봄 웜톤: 6.01%
여름 쿨톤: 93.99%
가을 웜톤: 0.00%
겨울 쿨톤: 0.00%

조이.jpg 파일의 결과:
봄 웜톤: 31.05%
여름 쿨톤: 68.92%
가을 웜톤: 0.03

In [10]:
file_results = {}

for filename in os.listdir(cool_tone_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        file_path = os.path.join(cool_tone_folder, filename)
        result = classify_image(file_path)
        file_results[filename] = result

# 결과 출력
for filename, result in file_results.items():
    print(f"{filename} 파일의 결과:\n{result}\n")

<ipython-input-4-3b249918057d>:7: RuntimeWarning: invalid value encountered in double_scalars
  spring_similarity = np.dot(color_vector, spring_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(spring_tone))
<ipython-input-4-3b249918057d>:8: RuntimeWarning: invalid value encountered in double_scalars
  summer_similarity = np.dot(color_vector, summer_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(summer_tone))
<ipython-input-4-3b249918057d>:9: RuntimeWarning: invalid value encountered in double_scalars
  fall_similarity = np.dot(color_vector, fall_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(fall_tone))
<ipython-input-4-3b249918057d>:10: RuntimeWarning: invalid value encountered in double_scalars
  winter_similarity = np.dot(color_vector, winter_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(winter_tone))


채영.jpeg 파일의 결과:
봄 웜톤: 12.63%
여름 쿨톤: 82.58%
가을 웜톤: 4.79%
겨울 쿨톤: 0.00%

태연.jpg 파일의 결과:
봄 웜톤: 11.30%
여름 쿨톤: 86.62%
가을 웜톤: 1.96%
겨울 쿨톤: 0.11%

한가인.jpg 파일의 결과:
봄 웜톤: 16.59%
여름 쿨톤: 72.50%
가을 웜톤: 10.25%
겨울 쿨톤: 0.67%

지수.jpg 파일의 결과:
봄 웜톤: 30.40%
여름 쿨톤: 69.59%
가을 웜톤: 0.01%
겨울 쿨톤: 0.00%

하니.jpg 파일의 결과:
봄 웜톤: 6.84%
여름 쿨톤: 89.47%
가을 웜톤: 3.69%
겨울 쿨톤: 0.00%

현아.jpg 파일의 결과:
봄 웜톤: 6.13%
여름 쿨톤: 92.50%
가을 웜톤: 1.36%
겨울 쿨톤: 0.01%

판빙빙.jpeg 파일의 결과:
봄 웜톤: 14.20%
여름 쿨톤: 83.39%
가을 웜톤: 0.80%
겨울 쿨톤: 1.61%

티파니.jpg 파일의 결과:
봄 웜톤: 10.12%
여름 쿨톤: 88.92%
가을 웜톤: 0.67%
겨울 쿨톤: 0.28%

찬미.jpg 파일의 결과:
봄 웜톤: 24.92%
여름 쿨톤: 75.07%
가을 웜톤: 0.00%
겨울 쿨톤: 0.00%

청하.jpg 파일의 결과:
봄 웜톤: 28.03%
여름 쿨톤: 71.48%
가을 웜톤: 0.49%
겨울 쿨톤: 0.00%

혜림.jpg 파일의 결과:
봄 웜톤: 39.67%
여름 쿨톤: 60.33%
가을 웜톤: 0.00%
겨울 쿨톤: 0.00%

유리.jpg 파일의 결과:
봄 웜톤: 8.37%
여름 쿨톤: 91.51%
가을 웜톤: 0.00%
겨울 쿨톤: 0.12%

정수정.jpg 파일의 결과:
봄 웜톤: 53.40%
여름 쿨톤: 46.45%
가을 웜톤: 0.14%
겨울 쿨톤: 0.01%

고현정.jpg 파일의 결과:
봄 웜톤: 4.98%
여름 쿨톤: 94.86%
가을 웜톤: 0.